# Exercises

## Get started with Netgen-Opencascade geometric modeling

* Consult the documentation, Section 4.4: https://docu.ngsolve.org/latest/i-tutorials/index.html#Geometric-modeling-and-mesh-generation
* Create some geometric models of your choice in 2D and 3D
* Generate meshes
* Give names to material and boundary regions. Execute `mesh.Materials()` and `mesh.Boundaries()` for verification.

### Example in 2d
Let's create a simple 2D geometry and mesh it. A rectangle with a circular hole in the middle.


In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

# import occ
from netgen.occ import *

# concept of workplanes: 
rect = WorkPlane().Rectangle(1, 1)
circ = WorkPlane().Circle(0.5, 0.5, 0.2)


# difference of two shapes
difference =  rect.Face() - circ.Face()

# draw the difference
Draw(difference)

# set the maxh for the edge 1:
difference.edges[1].maxh = 0.05


# so far the shapes are defined as topological entities of occ. Now we need to pass internally to netgen geometries.
geo = OCCGeometry(difference)

# from netgen we can generate a mesh:
mesh_2d = Mesh(geo.GenerateMesh(maxh=0.5))

# set a curvature of the mesh:
mesh_2d.Curve(3)

# draw the mesh:
Draw(mesh_2d)


# some more operations :

# difference = difference.Rotate(Axis((0,0,0),Z), 45)
# difference = difference.Move((0.5,0.5,0))

### Exercise 1
Reproduce the following geometry and mesh

![alt text](example1_todo.png)




In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

# import occ
from netgen.occ import *

# concept of workplanes: 
# rect = WorkPlane().Rectangle(1, 1)
y_c = 0.5
circ_a = WorkPlane().Circle(1.5, y_c, 0.5)
circ_b = WorkPlane().Circle(1.5, y_c, 0.25)

circ_c = WorkPlane().Circle(0.5,0.5, 0.2)
rec = WorkPlane().Rectangle(1,1)

down = rec.Face() - circ_c.Face()
disk = circ_a.Face() - circ_b.Face()
down = down.Rotate(Axis((0.5,0.5,0),Z), 45)
total = disk + down
total = total.Rotate(Axis((0.5, 0.5, 0),Z),90)

# down = down.Rotate(Axis((0.5, 0.5, 0), Z), -45)

Draw(total)


# # difference of two shapes
# difference =  rect.Face() - circ.Face()

# # draw the difference
# Draw(difference)

# # set the maxh for the edge 1:
# difference.edges[1].maxh = 0.05

In [135]:
rec = WorkPlane().Rectangle(0.5,1).Face()
rec = rec.Move((0.2, -0.5, 0))

test = WorkPlane().Circle(0,0, 0.9).Face() - \
            WorkPlane().Circle(0,0, 0.7).Face() +\
            WorkPlane().Circle(0,0, 0.5).Face() - \
            WorkPlane().Circle(0,0, 0.3).Face() - \
            rec

c_letter = WorkPlane().Circle(0,0, 0.5).Face() - \
            WorkPlane().Circle(0,0, 0.3).Face() - \
            rec


hor_line = WorkPlane().Rectangle(0.4, 0.2).Face()
hor_line_b = WorkPlane().Rectangle(0.4, 0.2).Face()

hor_a = hor_line.Move((0, 0.5, 0 ))
hor_b = hor_line_b.Move((0, 0, 0))
hor_c = hor_line.Move((0,1,0))

e_letter = WorkPlane().Rectangle(0.2,1).Face()+\
            hor_a + \
            hor_b + \
            hor_c


e_letter = e_letter.Move((0.4, -0.5, 0))

total = c_letter + e_letter
Draw(total)


geo = OCCGeometry(total)

# from netgen we can generate a mesh:
mesh = Mesh(geo.GenerateMesh(maxh=0.5))

# set a curvature of the mesh:
mesh.Curve(3)

Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
# from netgen.geom2d import SplineGeometry
# from netgen.meshing import Mesh as ngMesh

# # Create a rectangular geometry
# geo = SplineGeometry()
# p1 = (0, 0)
# p2 = (1, 0)
# p3 = (1, 0.5)
# p4 = (0, 0.5)

# geo.AddRectangle(p1, p2, p3)

# # Generate mesh from geometry
# mesh = ngMesh(geo.GenerateMesh(maxh=0.1))

# # Visualize or export the mesh as desired

# Draw(geo.Face())

### Example in 3d : Letters of the alphabet
We will create a fancy 3D geometry of the initial letter of name and last name. In my case E & B. 

![alt text](EB_letters.png) 

the starting point of the figure is the coordinate (0,0,0) in direction X.

for later purposes, we will move the initial point to (10,0,0). 

In [124]:
# use OCC geometry to write the initials of name and last name
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

# as always, we start with a workplane
# Start bottom left and move in X direction of 10
# move in X direction of 25
# create an arch starting at the last point
# rotate the coordinate system by 180 degrees
# ... and so on
# finally close the shape and extract the face

outer = WorkPlane()\
        .MoveTo(10, 0)\
        .Line(25)\
        .Arc(10, 180)\
        .Rotate(180)\
        .Arc(10, 180) \
        .Line(25) \
        .Close().Face()

innerE = WorkPlane().MoveTo(15, 5).Line(10).Rotate(90).Line(30).Rotate(90).Line(10).Rotate(90)  \
    .Line(12.50).Rotate(90).Line(7).Rotate(-90).Line(5).Rotate(-90).Line(7)\
    .Close().Face()


innerB1 = WorkPlane().MoveTo(32.5, 22.5).Arc(6, 180).Close().Face()

innerB2 = WorkPlane().MoveTo(32.5, 4.5).Arc(6, 180).Close().Face()

face_eb = outer-innerE-innerB1-innerB2


# select the edges with center of gravity less than 25 and name them "E"
face_eb.edges[X<25].name = "E"
face_eb.edges[X>=25].name = "B"

Draw(Compound(face_eb.edges[X < 25])) 

Draw(Compound(face_eb.edges[X >= 25]))


Draw(face_eb)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

In [136]:

#rot_eb = face_eb.Revolve(Axis((0, 0, 0), Y), 70) 
rot_eb = face_eb.Extrude(20)




# select all the faces that are no named and call them "Z"
for f in rot_eb.faces:
    if not f.name:
        f.name = "Z"

Draw(rot_eb)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

In [126]:
box = Box(Pnt(0, 0, 0), Pnt(10, 10, 10))
box.faces[:].name = "box"

rot_eb = rot_eb + box
solid_eb = Glue([rot_eb, box])
Draw(solid_eb)
solid_eb.solids[0].name = "cube"


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

In [ ]:


#solid_eb = rot_eb.MakeFillet(rot_eb.edges, 1)
#solid_eb = rot_eb 

Draw(solid_eb)



In [128]:

geo = OCCGeometry(solid_eb)
with TaskManager():
    mesh = Mesh(geo.GenerateMesh(maxh=4, grading=0.99))

mesh.Curve(3)
#fun = CF((z, 0, 0))

#mesh.deformation(fun)
#Draw(mesh)
#mesh.SplitElements_Alfeld()
Draw(mesh)



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Experiment with CoefficientFunctions
* Consult documentation https://docu.ngsolve.org/latest/i-tutorials/unit-1.2-coefficient/coefficientfunction.html
* Compute the volume and center of gravity of your geometries using the NGSolve - `Integrate(func, mesh)` functionality

In [129]:
# find the volume of the solid
cf = CoefficientFunction((1))
Integrate(cf, mesh)

print(mesh.GetMaterials())

#
lam = mesh.MaterialCF({"cube": 5}, default=1)
Draw(lam, mesh, "lambda")
#
#print(mats)

('cube', 'default')


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Work with GridFunctions
* Define an $H^1$ finite element space of some order on $\Omega = (0,1)^2$
* Define a `GridFunction` in this space, and interpolate $u(x,y) = \exp(-x^2 - y^2)$
* Compute the $L_2$-norm of the interpolation error $u - I_h u$.
* Study convergence of the interpolation error depending on the mesh-size and the polynomial order of the space (generate plots).
* Study convergence in the $H^1$-norm
* Find a way to compute $\int_{\partial \Omega} | \partial_n u- \partial_n I_h u) |^2$.

In [130]:
# to be sure on what I can set dirichlet boundary conditions print all the boundaries
print(mesh.GetBoundaries())
h1 = H1(mesh, order=1, dirichlet="Z")


# create a coefficient function and set it to the grid function
cf = CoefficientFunction(exp(-0.01*((x-15)**2 + (y-15)**2 + z**2)))
gf = GridFunction(h1)
gf.Set(cf)
Draw(gf, mesh, "gf")
print(Integrate(gf, mesh))




('box', 'box', 'box', 'box', 'box', 'box', 'box', 'box', 'B', 'Z', 'E', 'E', 'B', 'B', 'B', 'E', 'E', 'B', 'E', 'E', 'E', 'E', 'E', 'B', 'B')


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

1310.5511574897084


In [131]:
Draw(cf, mesh)
print(Integrate(cf, mesh))

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

1307.7010450600787


In [132]:
Draw(gf-cf, mesh, "error")
print(Integrate(Norm(gf-cf), mesh))

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

51.96881912421034


## Linear Algebra
* https://docu.ngsolve.org/latest/how_to/howto_linalg.html
* https://docu.ngsolve.org/latest/how_to/howto_numpy.html#
* Compute the Euklidean norm of the coefficient vector of the gridfunction set to the function above. Once use NGSolve functions, the other time use numpy

In [133]:
import numpy as np

# vec is the "view" of the grid function
# FV is the flat vector
# NumPy is the numpy array
npvec = gf.vec.FV().NumPy()

norm = np.linalg.norm(npvec)
print(norm)

4.131257324466285


## Experiments with `BilinearForm`s and `LinearForm`s

* Create a `GridFunction` gfu and interpolate $xy$ to it
* Define `LinearForm` $f : V \rightarrow {\mathbb R} : v \mapsto \int_\Omega 1 v \, dx$
* Compute $f(gfu)$ using `InnerProduct` of the vectors. Explain what you observe.
* Define a `BilinearForm` $A : V \times V \rightarrow {\mathbb R} : (u,v) \mapsto \int_\Omega u v + \nabla u \nabla v \, dx$
* Compute $\|u \|_{H^1}^2 = \| u \|_{L_2}^2 + \| \nabla u \|_{L_2}^2$ in two kinds: Once using `Integrate`, and then using the bilinear-form.

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
fes = H1(mesh, order=2, dirichlet=".*")
gfu = GridFunction(fes)
gfu.Set(CoefficientFunction((x*y)))
Draw(gfu)



u,v = fes.TnT()
f = LinearForm(fes)
f += 1*v*dx
f.Assemble()

print(InnerProduct(f.vec,gfu.vec))

a = BilinearForm(fes)
a+= (u*v + grad(u)*grad(v))*dx
a.Assemble()

# first way to compute the norm
l2_norm = Integrate((InnerProduct(gfu, gfu)), mesh) 
h1_seminorm = Integrate((InnerProduct(grad(gfu), grad(gfu))), mesh)

print(f"h1 norm squared {l2_norm + h1_seminorm}")


In [ ]:
# second way to calculate it is
h1_norm = InnerProduct(gfu.vec, a.mat * gfu.vec)
print(f"h1 norm squared {h1_norm}")

## Computing dual norms

* Define the `LinearForm` $f : V \rightarrow {\mathbb R} : v \mapsto \int_{\partial \Omega} v \, ds$
* The canonical norm for the linear-form is the dual norm
  
  $$
  \| f \|_{V^\ast} := \sup_{v \in V} \frac{f(v)}{\|v \|_V}.
  $$
  
  Proof that the supremum is attained for $w$ satisfying $(w,v)_V = f(v) \; \forall \, v \in V$, i.e. the solution of a variational problem.
* Compute the dual norm by replacing the space $V$ by a sequence of finite element spaces (refinement in $h$ and/or $p$). Try with norms $\| \cdot \|_{L_2}$ and $\| \cdot \|_{H^1}$.
